In [ ]:
# |default_exp mongo_episode

In [ ]:
from mongo import print_logs

print_logs(5)

{'_id': ObjectId('677a49b9a990da752d27b67d'), 'operation': 'delete', 'entite': 'critiques', 'desc': 'Patricia Martin', 'date': datetime.datetime(2025, 1, 5, 9, 58, 33, 718000)}
{'_id': ObjectId('677a49b9a990da752d27b67a'), 'operation': 'insert', 'entite': 'critiques', 'desc': 'Patricia Martin', 'date': datetime.datetime(2025, 1, 5, 9, 58, 33, 714000)}
{'_id': ObjectId('677a49b9a990da752d27b676'), 'operation': 'delete', 'entite': 'editeurs', 'desc': 'Gallimard', 'date': datetime.datetime(2025, 1, 5, 9, 58, 33, 688000)}
{'_id': ObjectId('677a49b9a990da752d27b673'), 'operation': 'insert', 'entite': 'editeurs', 'desc': 'Gallimard', 'date': datetime.datetime(2025, 1, 5, 9, 58, 33, 684000)}
{'_id': ObjectId('677a49b9a990da752d27b66f'), 'operation': 'delete', 'entite': 'auteurs', 'desc': 'Victor Hugo', 'date': datetime.datetime(2025, 1, 5, 9, 58, 33, 656000)}


# Episode entity

In [ ]:
# |export

from bson import ObjectId
from mongo import get_collection, get_DB_VARS, mongolog
from datetime import datetime

DATE_FORMAT = "%Y-%m-%dT%H:%M:%S.%f%z"


class Episode:
    def __init__(self, date: str, titre: str, collection_name: str = "episodes"):
        """
        Episode is a class that represents a generic Episode entity in the database.
        :param date: The date for this episode at the format "2024-12-22T09:59:39.000+00:00" parsed by "%Y-%m-%dT%H:%M:%S.%f%z".
        :param titre: The title of this episode.
        :param collection_name: The name of the collection. default: "episodes".
        """
        DB_HOST, DB_NAME, _ = get_DB_VARS()
        self.collection = get_collection(
            target_db=DB_HOST, client_name=DB_NAME, collection_name=collection_name
        )
        self.date = date
        self.titre = titre

    def exists(self) -> bool:
        """
        Check if the episode exists in the database.
        :return: True if the episode exists, False otherwise.
        """
        return (
            self.collection.find_one({"titre": self.titre, "date": self.date})
            is not None
        )

    def keep(self):
        """
        Keep the episode in the database.
        """
        if not self.exists():
            mongolog("insert", self.collection.name, f"{self.date} - {self.titre}")
            self.collection.insert_one({"titre": self.titre, "date": self.date})
        else:
            mongolog("update", self.collection.name, f"{self.date} - {self.titre}")

    def remove(self):
        """
        Remove the episode from the database.
        """
        self.collection.delete_one({"titre": self.titre, "date": self.date})
        mongolog("delete", self.collection.name, f"{self.date} - {self.titre}")

    def get_oid(self) -> ObjectId:
        """
        Get the object id of the episode.
        :return: The object id of the episode. (bson.ObjectId)
        None if does not exist.
        """
        document = self.collection.find_one({"titre": self.titre, "date": self.date})
        if document:
            return document["_id"]
        else:
            return None

    @staticmethod
    def get_date_from_string(date: str) -> datetime:
        """
        Get the datetime object from a string.
        :param date: The date string.
        :return: The datetime object.
        """
        return datetime.strptime(date, DATE_FORMAT)

    @staticmethod
    def get_string_from_date(date: datetime, format: str = None) -> str:
        """
        Get the string from a datetime object.
        :param date: The datetime object.
        :param format: The format of the string. default: None and DATE_FORMAT will be used.
        :return: The date string.
        """
        if format is not None:
            return date.strftime(format)
        else:
            return date.strftime(DATE_FORMAT)

# RSS_episode

In [ ]:
# |export


class RSS_episode(Episode):
    def __init__(self, date: str, titre: str):
        """
        RSS_episode is a class that represents an RSS episode in the database episodes.
        :param date: The date for this episode at the format "2024-12-22T09:59:39.000+00:00" parsed by "%Y-%m-%dT%H:%M:%S.%f%z".
        :param titre: The title of this episode.
        """
        super().__init__(date, titre)

In [ ]:
from datetime import datetime

now = datetime.now()

rss1 = RSS_episode(RSS_episode.get_string_from_date(now), "test RSS 1")
print(f"Est-ce que l episode existe ? {rss1.exists()}")

rss1.keep()
print(f"et maintenant, st-ce que rss1 existe ? {rss1.exists()}")
print(f"et voici l'id de rss1 : {rss1.get_oid()}")

rss1.remove()
print(f"après nettoyage, est-ce que rss1 existe ? {rss1.exists()}")
print(f"et son oid : {rss1.get_oid()}")

Est-ce que l episode existe ? False
et maintenant, st-ce que rss1 existe ? True
et voici l'id de rss1 : 6783927d24dc07fa7b62ab27
après nettoyage, est-ce que rss1 existe ? False
et son oid : None


In [ ]:
print_logs(5)

{'_id': ObjectId('6783927d24dc07fa7b62ab29'), 'operation': 'delete', 'entite': 'episodes', 'desc': '2025-01-12T10:59:25.461228 - test RSS 1', 'date': datetime.datetime(2025, 1, 12, 10, 59, 25, 490000)}
{'_id': ObjectId('6783927d24dc07fa7b62ab26'), 'operation': 'insert', 'entite': 'episodes', 'desc': '2025-01-12T10:59:25.461228 - test RSS 1', 'date': datetime.datetime(2025, 1, 12, 10, 59, 25, 478000)}
{'_id': ObjectId('6783920b24dc07fa7b62ab22'), 'operation': 'delete', 'entite': 'episodes', 'desc': {'titre': 'test RSS 1', 'date': '2025-01-12T10:57:31.901774'}, 'date': datetime.datetime(2025, 1, 12, 10, 57, 31, 937000)}
{'_id': ObjectId('6783920b24dc07fa7b62ab1f'), 'operation': 'insert', 'entite': 'episodes', 'desc': '2025-01-12T10:57:31.901774 - test RSS 1', 'date': datetime.datetime(2025, 1, 12, 10, 57, 31, 921000)}
{'_id': ObjectId('677a49b9a990da752d27b67d'), 'operation': 'delete', 'entite': 'critiques', 'desc': 'Patricia Martin', 'date': datetime.datetime(2025, 1, 5, 9, 58, 33, 7180

# extract py

In [ ]:
from nbdev.export import nb_export

nb_export("py mongo helper episodes.ipynb", ".")